In [233]:
import pandas as pd
import numpy as np
import os
import string
from datetime import tzinfo, timedelta, datetime

os.getcwd()
import glob



In [234]:
def convert_mm(value):
    #length in mm transformed to float
    try:
        new_value = value.replace(' mm','')
        return float(new_value)    
    except AttributeError:
        return np.NaN

In [235]:
def convert_exposure(value):
    #length in mm transformed to float
    try:
        list_string=value.split('/')
        if (len(list_string)==1):
            return float(value)
        else:
            return float(list_string[0])/float(list_string[1])
    except AttributeError:
        #print ('WTF',value)
        return np.NaN
    #return float(0)
        

In [236]:
def remove_full_path(value):
    #all pictures originate from a Pictures folder, remove the full path
    #including Pictures
    try:
        list_string=value.split('/Pictures')
        return list_string[1]
    except AttributeError:
        return np.NaN

In [237]:
def funcDayPeriod(filename):
    search_night=["bei Nacht",'Sternenhimmel',"Mondschein","Mond"]
    search_night_special=["Lumiere","euerwerk","Firework","Freudenfeuer"]
    search_twilight=["bei anbrechender Nacht","Abenddaemmerung","Morgendaemmerung","Sonnenaufgang"]
    if any (x in filename for x in search_night):
        return -1
    elif any(x in filename for x in search_night_special):
         return 0
    elif any(x in filename for x in search_twilight):
        return 3
    elif "bei Sonnenuntergang" in filename:
        return 2
    else:
        return 1

In [238]:
def funcComposite(filename):
    if "Panorama." in filename:
        return 1
    else:
        return 0

In [239]:
def funcMovie(filename):
    search_file_names=[".JPG",".jpg",".JPEG",".jpeg"]
    if any(x in filename for x in search_file_names):
        return 0
    else:
        return 1

In [240]:
def funcTimeZoneCorrection(value,filename,modelname):
    print("date",value,'of file',filename,'of model',modelname)
    

In [241]:
dataframe_list=[]

#push-back day related periods, 1 for day,2 for sunset,3 for dusk and daw,sunrise, 0 for night
for input in range(2006,2020):
    df=pd.read_csv("/Users/matthiasweber/outputEXIFcsvs/PhotoEXIFDataDayType_%s.csv"%(str(input)))
    #push back year index as additional list 
    #print(df.columns,df.head)
    #print(df.dtypes)
    df['SourceFile']=df['SourceFile'].apply(remove_full_path)    
    df['FocalLength']=df['FocalLength'].apply(convert_mm) 
    df['ExposureTime']=df['ExposureTime'].apply(convert_exposure)
    df['year']=int(input)
    df['dayperiod']=df.apply(lambda x: funcDayPeriod(x['SourceFile']), axis=1)
    df['CompositePhoto']=df.apply(lambda x: funcComposite(x['SourceFile']), axis=1)
    df['Movie']=df.apply(lambda x: funcMovie(x['SourceFile']), axis=1)
    df.rename(columns={'DateTimeOriginal':'Date','FocalLength':'FocalLength in mm'},inplace=True) 
    total_rows=df['SourceFile'].count()
    #print('did something happen')
    fourth = df.loc[df['SourceFile'].str.contains("Abenddaemmerung")]
    #fourth['dayperiod']=int(3)
    total_rows_fourth=fourth['SourceFile'].count()
    dataframe_list.append(df)
    print("rows per year", input,total_rows)
 
combine_all_data=pd.DataFrame(dataframe_list[0]) 
print(combine_all_data.head)
print("now for the rest")
dataframe_list.pop(0)
for data in dataframe_list:
    print(data.head)
    combine_all_data=pd.concat([combine_all_data,data],ignore_index=True,sort=False)
    print(combine_all_data.head)
    
combine_all_data.to_csv(r'EXIFDataCombinedCameras_2006_to_2019.csv')
    

rows per year 2006 576
rows per year 2007 1496
rows per year 2008 2206
rows per year 2009 4195
rows per year 2010 5837
rows per year 2011 4222
rows per year 2012 5270
rows per year 2013 7805
rows per year 2014 9304
rows per year 2015 8038
rows per year 2016 10196
rows per year 2017 11875
rows per year 2018 13506
rows per year 2019 7020
<bound method NDFrame.head of                                             SourceFile  \
0    /2006/060710 - London/P7100001 - Basel, Easyje...   
1    /2006/060710 - London/P7100002 - London, Leice...   
2    /2006/060710 - London/P7100003 - London, House...   
3    /2006/060710 - London/P7100004 - London, Westm...   
4    /2006/060710 - London/P7100005 - London, St. M...   
..                                                 ...   
571  /2006/Private/P7160164 - London, Matthias im H...   
572  /2006/Private/P7180300 - London, Heidrun, Matt...   
573  /2006/Private/PA050036 - Zuerich, Anne in ETH ...   
574  /2006/Private/PA050040 - Zuerich, Sigi & Anne .

<bound method NDFrame.head of                                               SourceFile  \
0      /2006/060710 - London/P7100001 - Basel, Easyje...   
1      /2006/060710 - London/P7100002 - London, Leice...   
2      /2006/060710 - London/P7100003 - London, House...   
3      /2006/060710 - London/P7100004 - London, Westm...   
4      /2006/060710 - London/P7100005 - London, St. M...   
...                                                  ...   
48944  /2015/151108-151115 - Berlin & Deutschland/PB1...   
48945  /2015/151108-151115 - Berlin & Deutschland/PB1...   
48946  /2015/151108-151115 - Berlin & Deutschland/PB1...   
48947  /2015/151108-151115 - Berlin & Deutschland/PB1...   
48948  /2015/151108-151115 - Berlin & Deutschland/PB1...   

                             Date  FocalLength in mm  ExposureTime  \
0      Jul 10 Mon ,2006, 12:36:06                6.3      0.001250   
1      Jul 10 Mon ,2006, 16:52:49                6.3      0.003125   
2      Jul 10 Mon ,2006, 17:17:26      